<h1 style="font-size:3rem;color:orange;">Eleições 2022 - Estatísticas do Banco de Dados</h1>

AVISO de Licença de Uso e Reserva de Direitos Autorais BSD 3 Clause. Copyright (c) 2022 por John Robson <john.robson@msn.com> (PIX)

Todo o Código fonte e demais arquivos estão sobre a Licença BSD 3 Clause. Modificações, Redistribuição, uso Comercial são permitidos, sempre informando
esse aviso de direitos autorais. Repositório oficial: https://github.com/JohnRobson/Eleicoes2022

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import timeit
import ast
from sqlalchemy import select, func

import matplotlib.pylab as plt
%matplotlib inline

from IPython.display import Markdown, display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>div.output_scroll { height: 100%; }</style>"))

from bd_urnas import engine, Base, Session, Urnas
from src_config import federacao, estados_federacao
session = Session()

def printmd(string): # convert markdown para HTML
    display(Markdown(string))

## Sobre as Urnas Usadas nas Eleições

In [3]:
printmd(f'''Total de Urnas Inseridas Usadas nas Eleições:

Durante o Primeiro Turno: **{session.execute("SELECT COUNT(id) FROM urnas WHERE turno=1").scalar()}** urnas.

Durante o Segundo Turno: **{session.execute("SELECT COUNT(id) FROM urnas WHERE turno=2").scalar()}** urnas.''')

Total de Urnas Inseridas Usadas nas Eleições:

Durante o Primeiro Turno: **472024** urnas.

Durante o Segundo Turno: **472027** urnas.

In [4]:
df = pd.read_sql_query('SELECT modelUrna AS "Modelo da Urna", COUNT(*) AS "Quantidade" from urnas GROUP BY modelUrna ORDER BY modelUrna', session.bind)

printmd('''obs: **UE0000** é o código para urnas sem o arquivo Log da urna e **UE1111** são os Logs de urna que não identificavam o modelo da urna.

No segundo turno, apenas o Pará enviou o arquivo BU ('.bu' ou '.busa') de 5 Urnas, mas não tem o arquivo de Log da Urna ('.logjez' ou '.logsajez') de um total de 18235 urnas.''')

df

obs: **UE0000** é o código para urnas sem o arquivo Log da urna e **UE1111** são os Logs de urna que não identificavam o modelo da urna.

No segundo turno, apenas o Pará enviou o arquivo BU ('.bu' ou '.busa') de 5 Urnas, mas não tem o arquivo de Log da Urna ('.logjez' ou '.logsajez') de um total de 18235 urnas.

,Modelo da Urna,Quantidade
0,UE0000,62
1,UE1111,10
2,UE2009,94881
3,UE2010,188155
4,UE2011,54425
5,UE2013,49658
6,UE2015,171529
7,UE2020,385331


## Votos Totais em Todos Estados

In [5]:
for i in [1, 2]:
    df = pd.read_sql_query(f'SELECT regiao, estado, SUM(vPresA22) AS "PresA22", SUM(vPresB13) AS "PresB13", SUM(vPresNulo) AS "PresNulo",\
    SUM(vPresBranco) AS "PresBranco", SUM(vGovA) AS "GovA", SUM(vGovB) AS "GovB", SUM(vGovNulo) AS "GovNulo", SUM(vGovBranco) AS "GovBranco"\
    FROM urnas WHERE turno={i} GROUP BY estado ORDER BY regiao', session.bind)
    df.fillna('', inplace=True) # substitui remove campos "NaN"

    printmd(f'Total de Votos para Presidente e Governador durante o 1o Turno em Todos os estados e exterior durante o {i}o Turno')
    df

    votos_presidente = df[['PresA22', 'PresB13', 'PresNulo', 'PresBranco']].sum()
    votos_presidente

    printmd(f'Total de Votos (Presidente) no {i}o Turno: **{votos_presidente.sum()}**')

Total de Votos para Presidente e Governador durante o 1o Turno em Todos os estados e exterior durante o 1o Turno

,regiao,estado,PresA22,PresB13,PresNulo,PresBranco,GovA,GovB,GovNulo,GovBranco
0,CO,DF,910397,649534,32163,25162,832633.0,434587.0,86099.0,65969.0
1,CO,GO,1920203,1454723,82589,48539,1806892.0,516579.0,166471.0,152864.0
2,CO,MS,794206,588323,30509,17651,384275.0,361981.0,61130.0,53082.0
3,CO,MT,1102866,633748,30612,18680,1114549.0,267172.0,162124.0,98676.0
4,EX,ZZ,122548,138933,4645,4857,,,,
5,N,AC,275582,129022,10322,4664,242100.0,103265.0,21077.0,7761.0
6,N,AM,880198,1019684,37525,19570,819784.0,401817.0,138218.0,58257.0
7,N,AP,187621,197382,7392,3289,176208.0,222168.0,21795.0,6494.0
8,N,PA,1884673,2443730,68473,40902,1201079.0,3117276.0,258639.0,99146.0
9,N,RO,581306,261749,14242,9350,315035.0,330656.0,44643.0,30765.0


PresA22       51072345
PresB13       57259504
PresNulo       3487869
PresBranco     1964779
dtype: int64

Total de Votos (Presidente) no 1o Turno: **113784497**

Total de Votos para Presidente e Governador durante o 1o Turno em Todos os estados e exterior durante o 2o Turno

,regiao,estado,PresA22,PresB13,PresNulo,PresBranco,GovA,GovB,GovNulo,GovBranco
0,CO,DF,1041331,729295,38203,29663,,,,
1,CO,GO,2193041,1542115,81585,43610,,,,
2,CO,MS,880606,599547,51658,18062,612113.0,808210.0,88228.0,39059.0
3,CO,MT,1216730,652786,27398,16317,,,,
4,EX,ZZ,145264,152905,6079,5900,,,,
5,N,AC,287750,121566,6722,4722,,,,
6,N,AM,961741,1004991,79549,21594,1039192.0,795098.0,157882.0,72907.0
7,N,AP,200547,189918,6200,4018,,,,
8,N,PA,2073895,2509084,75159,43602,,,,
9,N,RO,633236,262904,20949,9428,415278.0,458370.0,31812.0,20103.0


PresA22       58206354
PresB13       60345999
PresNulo       3930765
PresBranco     1769678
dtype: int64

Total de Votos (Presidente) no 2o Turno: **124252796**

In [6]:
for i in range(11): print(f'* Urnas que os candidatos obteviveram até {i} Votos: Bolsonaro ({session.execute(f"SELECT COUNT(id) FROM urnas WHERE turno=1 AND vPresA22 <= {i}").scalar()}), Lula ({session.execute(f"SELECT COUNT(id) FROM urnas WHERE turno=1 AND vPresB13 <= {i}").scalar()})')

* Urnas que os candidatos obteviveram até 0 Votos: Bolsonaro (161), Lula (1)
* Urnas que os candidatos obteviveram até 1 Votos: Bolsonaro (417), Lula (5)
* Urnas que os candidatos obteviveram até 2 Votos: Bolsonaro (746), Lula (13)
* Urnas que os candidatos obteviveram até 3 Votos: Bolsonaro (1134), Lula (19)
* Urnas que os candidatos obteviveram até 4 Votos: Bolsonaro (1571), Lula (35)
* Urnas que os candidatos obteviveram até 5 Votos: Bolsonaro (2071), Lula (48)
* Urnas que os candidatos obteviveram até 6 Votos: Bolsonaro (2628), Lula (68)
* Urnas que os candidatos obteviveram até 7 Votos: Bolsonaro (3283), Lula (95)
* Urnas que os candidatos obteviveram até 8 Votos: Bolsonaro (3939), Lula (122)
* Urnas que os candidatos obteviveram até 9 Votos: Bolsonaro (4614), Lula (152)
* Urnas que os candidatos obteviveram até 10 Votos: Bolsonaro (5402), Lula (183)


In [7]:
for i in range(11): print(f'* Urnas que os candidatos obteviveram até {i} Votos: Bolsonaro ({session.execute(f"SELECT COUNT(id) FROM urnas WHERE turno=2 AND vPresA22 <= {i}").scalar()}), Lula ({session.execute(f"SELECT COUNT(id) FROM urnas WHERE turno=2 AND vPresB13 <= {i}").scalar()})')

* Urnas que os candidatos obteviveram até 0 Votos: Bolsonaro (143), Lula (4)
* Urnas que os candidatos obteviveram até 1 Votos: Bolsonaro (352), Lula (8)
* Urnas que os candidatos obteviveram até 2 Votos: Bolsonaro (617), Lula (13)
* Urnas que os candidatos obteviveram até 3 Votos: Bolsonaro (883), Lula (18)
* Urnas que os candidatos obteviveram até 4 Votos: Bolsonaro (1191), Lula (28)
* Urnas que os candidatos obteviveram até 5 Votos: Bolsonaro (1590), Lula (44)
* Urnas que os candidatos obteviveram até 6 Votos: Bolsonaro (2001), Lula (58)
* Urnas que os candidatos obteviveram até 7 Votos: Bolsonaro (2449), Lula (74)
* Urnas que os candidatos obteviveram até 8 Votos: Bolsonaro (2977), Lula (101)
* Urnas que os candidatos obteviveram até 9 Votos: Bolsonaro (3559), Lula (124)
* Urnas que os candidatos obteviveram até 10 Votos: Bolsonaro (4117), Lula (160)


## Usando Pandas DataFrames

In [9]:
# Importando Banco de Dados para Python Pandas Dataframe
bd = pd.read_sql_query('SELECT id, turno, regiao, estado, municipio, zona, local, secao, vPresA22, vPresB13, vPresNulo, vPresBranco, vGovA,\
vGovB, vGovNulo, vGovBranco, qEleitAptos, qComparecimento, qEleitLibCodigo, qEleitBiometrico, modelUrna, arquivoBU FROM urnas', session.bind)

In [10]:
# Criação de Colunas Extas para Análises
bd['vGovTotal'] = bd['vGovA'] + bd['vGovB']
bd

,id,turno,regiao,estado,municipio,zona,local,secao,vPresA22,vPresB13,...,vGovB,vGovNulo,vGovBranco,qEleitAptos,qComparecimento,qEleitLibCodigo,qEleitBiometrico,modelUrna,arquivoBU,vGovTotal
0,1,1,CO,DF,97012,2,1465,272,146,127,...,99.0,7.0,7.0,395,327,22,289,UE2020,o00406-9701200020272.bu,229.0
1,2,1,CO,DF,97012,14,1619,357,113,151,...,131.0,11.0,6.0,373,321,10,304,UE2020,o00406-9701200140357.bu,231.0
2,3,1,CO,DF,97012,4,1520,342,174,114,...,49.0,14.0,26.0,397,332,23,316,UE2013,o00406-9701200040342.bu,185.0
3,4,1,CO,DF,97012,14,1015,6,180,73,...,67.0,7.0,7.0,363,287,9,265,UE2020,o00406-9701200140006.bu,187.0
4,5,1,CO,DF,97012,14,1031,281,131,137,...,113.0,10.0,16.0,364,312,16,287,UE2020,o00406-9701200140281.bu,215.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944046,944047,2,EX,ZZ,29696,1,1040,1619,69,378,...,NaN,NaN,NaN,761,457,0,0,UE2009,o00407-2969600011619.bu,NaN
944047,944048,2,EX,ZZ,29696,1,1031,1601,103,136,...,NaN,NaN,NaN,607,253,0,0,UE2009,o00407-2969600011601.bu,NaN
944048,944049,2,EX,ZZ,29696,1,1031,1597,57,122,...,NaN,NaN,NaN,605,184,0,0,UE2009,o00407-2969600011597.bu,NaN
944049,944050,2,EX,ZZ,29696,1,1031,1594,68,225,...,NaN,NaN,NaN,551,301,0,0,UE2009,o00407-2969600011594.bu,NaN


In [11]:
printmd(f'Modelos de Urnas Usadas nas eleições, e quantidade por modelo de urna.')
bd[bd['turno'] == 2].groupby(['modelUrna'])['modelUrna'].count()

printmd(f'Modelos de Urnas Usadas nas eleições, e quantidade por modelo para cada turno.')
bd.groupby(['turno', 'modelUrna'])['modelUrna'].count()

Modelos de Urnas Usadas nas eleições, e quantidade por modelo de urna.

modelUrna
UE0000         5
UE2009     48245
UE2010     93790
UE2011     26889
UE2013     24672
UE2015     85735
UE2020    192691
Name: modelUrna, dtype: int64

Modelos de Urnas Usadas nas eleições, e quantidade por modelo para cada turno.

turno  modelUrna
1      UE0000           57
       UE1111           10
       UE2009        46636
       UE2010        94365
       UE2011        27536
       UE2013        24986
       UE2015        85794
       UE2020       192640
2      UE0000            5
       UE2009        48245
       UE2010        93790
       UE2011        26889
       UE2013        24672
       UE2015        85735
       UE2020       192691
Name: modelUrna, dtype: int64

In [12]:
printmd(f'Estados com votos para Governador no Segundo Turno')
bd[bd['turno'] == 2]

Estados com votos para Governador no Segundo Turno

,id,turno,regiao,estado,municipio,zona,local,secao,vPresA22,vPresB13,...,vGovB,vGovNulo,vGovBranco,qEleitAptos,qComparecimento,qEleitLibCodigo,qEleitBiometrico,modelUrna,arquivoBU,vGovTotal
472024,472025,2,CO,DF,97012,9,1082,88,108,80,...,NaN,NaN,NaN,232,191,14,167,UE2020,o00407-9701200090088.bu,NaN
472025,472026,2,CO,DF,97012,9,1287,397,140,111,...,NaN,NaN,NaN,307,264,12,246,UE2020,o00407-9701200090397.bu,NaN
472026,472027,2,CO,DF,97012,9,1147,162,167,109,...,NaN,NaN,NaN,329,284,27,283,UE2020,o00407-9701200090162.bu,NaN
472027,472028,2,CO,DF,97012,9,1147,150,156,108,...,NaN,NaN,NaN,325,277,18,273,UE2020,o00407-9701200090150.bu,NaN
472028,472029,2,CO,DF,97012,9,1384,429,175,157,...,NaN,NaN,NaN,395,341,19,259,UE2020,o00407-9701200090429.bu,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944046,944047,2,EX,ZZ,29696,1,1040,1619,69,378,...,NaN,NaN,NaN,761,457,0,0,UE2009,o00407-2969600011619.bu,NaN
944047,944048,2,EX,ZZ,29696,1,1031,1601,103,136,...,NaN,NaN,NaN,607,253,0,0,UE2009,o00407-2969600011601.bu,NaN
944048,944049,2,EX,ZZ,29696,1,1031,1597,57,122,...,NaN,NaN,NaN,605,184,0,0,UE2009,o00407-2969600011597.bu,NaN
944049,944050,2,EX,ZZ,29696,1,1031,1594,68,225,...,NaN,NaN,NaN,551,301,0,0,UE2009,o00407-2969600011594.bu,NaN
